In [1]:
## Fast LSTM implementation backed by CuDNN. Can only be run on GPU, with the TensorFlow backend.

from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
t_in = Input(shape=(None,))
t = t_in

char2id = {'如': 2,'何': 3,'演': 4,'好': 5}
char_size = 64

In [3]:
mask = Lambda(lambda x: K.cast(K.greater(K.expand_dims(x, 2), 0), 'float32'))(t)
mask

<tf.Tensor 'lambda_1/Cast:0' shape=(?, ?, 1) dtype=float32>

In [1]:
import numpy as np
xs = np.array([
        [[2,3], [3,4], [4,5], [5,6], [5,7]],
               [[2,3], [3,4], [4,5], [5,6], [5,7]]
              ]
               )
res = np.array([[0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1]])
xs.shape #  num_sample,time_step,num_fea

(2, 5, 2)

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM
model = Sequential()
model.add(LSTM(100 // 2, return_sequences=True))
model.compile('adam')

In [6]:
model.predict(xs).shape  

(2, 5, 50)

In [4]:

t = Embedding(len(char2id)+2, char_size)(t) # 0: padding, 1: unk
t = Dropout(0.25)(t)
t = Lambda(lambda x: x[0] * x[1])([t, mask])
# t = Bidirectional(LSTM(char_size/2, return_sequences=True))(t)
lstm = LSTM(char_size/2, return_sequences=True)
blstm = Bidirectional(lstm)
t = blstm(t)
t



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


TypeError: Value passed to parameter 'shape' has DataType float32 not in list of allowed values: int32, int64